In [ ]:
from sklearn.model_selection import GridSearchCV
class ChangeDetector:
    """
    Computes change metrics between two images including SAM, CVA, NDVI, NDWI, NDBI, and EVI.
    Now includes all 13 Sentinel-2 bands.
    """
    # Mapping for all 13 Sentinel-2 bands.
    band_to_index = {
        'B1': 0,    # Coastal Aerosol
        'B2': 1,    # Blue
        'B3': 2,    # Green
        'B4': 3,    # Red
        'B5': 4,    # Vegetation Red Edge 1
        'B6': 5,    # Vegetation Red Edge 2
        'B7': 6,    # Vegetation Red Edge 3
        'B8': 7,    # Near-Infrared (NIR)
        'B8A': 8,   # Narrow NIR
        'B9': 9,    # Water Vapour
        'B10': 10,  # Cirrus
        'B11': 11,  # SWIR 1
        'B12': 12   # SWIR 2
    }

    # Default bands and their significance for different landcover types.
    selected_bands = {
        'Vegetation': {
            'bands': ['B4', 'B8A'],
            'description': ("B4 (Red) and B8A (NIR) are critical for NDVI, indicating vegetation vigor "
                            "and health; a significant drop in NDVI (with high SAM/CVA) may indicate deforestation or vegetation stress.")
        },
        'Urban': {
            'bands': ['B11', 'B12', 'B4'],
            'description': ("SWIR bands (B11, B12) and B4 (Red) are used in NDBI; a rise in NDBI with high CVA can indicate urban expansion or structural changes.")
        },
        'Water': {
            'bands': ['B3', 'B8A'],
            'description': ("B3 (Green) and B8A (NIR) are used for NDWI; an increase in NDWI together with a spectral shift (SAM) may indicate water body expansion or improved water quality.")
        }
    }

    def __init__(self, threshold_scaler=THRESHOLD_SCALER):
        self.threshold_scaler = threshold_scaler

    def _extract_band_list(self, bands):
        if isinstance(bands, dict):
            return bands.get('bands', [])
        return bands

    def compute_sam(self, image1, image2, bands):
        band_list = self._extract_band_list(bands)
        print("compute_sam - Using bands:", band_list)
        indices = [self.band_to_index[b] for b in band_list]
        print("compute_sam - Band indices:", indices)
        img1 = image1[:, :, indices]
        img2 = image2[:, :, indices]
        dot = np.sum(img1 * img2, axis=2)
        print("compute_sam - Dot product sample (first row):", dot[0, :5])
        norm1 = np.linalg.norm(img1, axis=2)
        norm2 = np.linalg.norm(img2, axis=2)
        print("compute_sam - Norm1 mean:", np.mean(norm1), "Norm2 mean:", np.mean(norm2))
        cos_angle = dot / (norm1 * norm2 + 1e-8)
        cos_angle = np.clip(cos_angle, -1, 1)
        sam = np.arccos(cos_angle)
        print("compute_sam - SAM (min, max):", np.min(sam), np.max(sam))
        return sam

    def compute_cva(self, image1, image2, bands):
        band_list = self._extract_band_list(bands)
        print("compute_cva - Using bands:", band_list)
        indices = [self.band_to_index[b] for b in band_list]
        print("compute_cva - Band indices:", indices)
        img1 = image1[:, :, indices]
        img2 = image2[:, :, indices]
        diff = img2 - img1
        cva = np.linalg.norm(diff, axis=2)
        print("compute_cva - CVA (min, max):", np.min(cva), np.max(cva))
        return cva

    def apply_otsu(self, change_image, change_type="Generic"):
        """
        Apply global Otsu's thresholding (scaled) to generate a binary mask.
        NOTE: No differentiation is done for Vegetation, Urban, or Water.
        """
        thresh = filters.threshold_otsu(change_image)
        print(f"Global Otsu threshold: {thresh:.3f}")
        scaled_thresh = self.threshold_scaler * thresh
        print(f"Scaled threshold: {scaled_thresh:.3f}")
        binary_mask = change_image > scaled_thresh
        return binary_mask

    def compute_ndvi(self, image, red_index=band_to_index['B4'], nir_index=band_to_index['B8']):
        red = image[:, :, red_index].astype(float)
        nir = image[:, :, nir_index].astype(float)
        ndvi = (nir - red) / (nir + red + 1e-8)
        print("compute_ndvi - NDVI (mean):", np.mean(ndvi), " (min, max):", np.min(ndvi), np.max(ndvi))
        return ndvi

    def compute_ndwi(self, image, green_index=band_to_index['B3'], nir_index=band_to_index['B8']):
        green = image[:, :, green_index].astype(float)
        nir = image[:, :, nir_index].astype(float)
        ndwi = (green - nir) / (green + nir + 1e-8)
        print("compute_ndwi - NDWI (mean):", np.mean(ndwi), " (min, max):", np.min(ndwi), np.max(ndwi))
        return ndwi

    def compute_ndbi(self, image, swir_index=band_to_index['B11'], nir_index=band_to_index['B8']):
        swir = image[:, :, swir_index].astype(float)
        nir = image[:, :, nir_index].astype(float)
        ndbi = (swir - nir) / (swir + nir + 1e-8)
        print("compute_ndbi - NDBI (mean):", np.mean(ndbi), " (min, max):", np.min(ndbi), np.max(ndbi))
        return ndbi

    def compute_evi(self, image, red_index=band_to_index['B4'], nir_index=band_to_index['B8'], blue_index=band_to_index['B2']):
        red = image[:, :, red_index].astype(float)
        nir = image[:, :, nir_index].astype(float)
        blue = image[:, :, blue_index].astype(float)
        evi = 2.5 * (nir - red) / (nir + 6*red - 7.5*blue + 1)
        print("compute_evi - EVI (mean):", np.mean(evi), " (min, max):", np.min(evi), np.max(evi))
        return evi

    def generate_description(self, landcover, avg_diff, pos_thresh=0.1, neg_thresh=-0.1):
        percent_change = f"{avg_diff*100:.1f}%"
        extra_info = ""
        if landcover in self.selected_bands:
            extra_info = " " + self.selected_bands[landcover]['description']

        if landcover == 'Vegetation':
            if avg_diff > pos_thresh:
                return (f"Vegetation increased by {percent_change} (greening). "
                        "This is indicated by a rise in NDVI and confirmed by a spectral shift (high SAM) "
                        "and significant magnitude change (high CVA)." + extra_info)
            elif avg_diff < neg_thresh:
                return (f"Vegetation decreased by {percent_change} (deforestation). "
                        "A drop in NDVI values along with a marked spectral shift (high SAM) indicates vegetation loss." + extra_info)
            else:
                return (f"Minor vegetation change ({percent_change}). " + extra_info)
        elif landcover == 'Urban':
            if avg_diff > pos_thresh:
                return (f"Urban areas expanded by {percent_change}. "
                        "An increase in NDBI, combined with high CVA, suggests urban growth." + extra_info)
            elif avg_diff < neg_thresh:
                return (f"Urban areas reduced by {percent_change}. "
                        "A decrease in NDBI values indicates a reduction in urban structures." + extra_info)
            else:
                return (f"Minor urban change ({percent_change}). " + extra_info)
        elif landcover == 'Water':
            if avg_diff > pos_thresh:
                return (f"Water bodies increased by {percent_change}. "
                        "A rise in NDWI along with spectral changes implies expansion of water areas." + extra_info)
            elif avg_diff < neg_thresh:
                return (f"Water bodies decreased by {percent_change}. "
                        "A drop in NDWI values indicates a reduction in water coverage." + extra_info)
            else:
                return (f"Minor water change ({percent_change}). " + extra_info)
        return "No significant change"

# =============================================================================
# RegionRefiner Class
# =============================================================================
class RegionRefiner:
    """
    Refines binary change masks using distance transform and watershed segmentation,
    and extracts boundaries using contour detection.
    """
    def __init__(self, min_size=MIN_REGION_AREA):
        self.min_size = min_size

    def refine(self, binary_mask):
        from scipy import ndimage as ndi
        from skimage import morphology, segmentation
        from skimage.measure import label, regionprops_table

        distance = ndi.distance_transform_edt(binary_mask)
        local_max = morphology.h_maxima(distance, 0.1)
        markers = ndi.label(local_max)[0]
        ws_labels = segmentation.watershed(-distance, markers, mask=binary_mask)
        labeled = label(ws_labels)
        props = regionprops_table(labeled, properties=['label', 'area'])
        labels_arr = np.array(props['label'])
        areas_arr = np.array(props['area'])
        valid_labels = labels_arr[areas_arr >= self.min_size]
        mapping = np.zeros(labeled.max() + 1, dtype=int)
        new_labels = np.arange(1, len(valid_labels) + 1)
        mapping[valid_labels] = new_labels
        refined = mapping[labeled]
        return refined

    def extract_boundaries(self, refined_mask):
        boundaries = []
        for reg in np.unique(refined_mask):
            if reg == 0:
                continue
            mask = (refined_mask == reg).astype(float)
            contours = find_contours(mask, 0.5)
            boundaries.extend(contours)
        return boundaries

# =============================================================================
# Visualizer Class
# =============================================================================
class Visualizer:
    """
    Provides methods for professional, uniform visualization of results and detailed report display.
    """
    @staticmethod
    def professional_plot_setup():
        import matplotlib.pyplot as plt
        plt.rcParams.update({
            "font.size": FONT_SIZE,
            "font.family": "sans-serif",
            "axes.titlesize": TITLE_SIZE,
            "axes.titleweight": "bold",
            "figure.dpi": FIGURE_DPI
        })

    @staticmethod
    def display_before_after(rgb_before, rgb_after):
        import matplotlib.pyplot as plt
        Visualizer.professional_plot_setup()
        fig, axes = plt.subplots(1, 2, figsize=FIGURE_SIZE_COMMON)
        axes[0].imshow(rgb_before)
        axes[0].set_title("Before")
        axes[0].axis('off')
        axes[1].imshow(rgb_after)
        axes[1].set_title("After")
        axes[1].axis('off')
        plt.tight_layout()
        plt.show()

    @staticmethod
    def display_difference_maps(ndvi_diff, ndwi_diff, ndbi_diff, evi_diff, sam_map, cva_map):
        import matplotlib.pyplot as plt
        Visualizer.professional_plot_setup()
        fig, axes = plt.subplots(2, 3, figsize=FIGURE_SIZE_COMMON)
        diff_maps = [(ndvi_diff, "NDVI Difference", ndvi_cmap),
                     (ndwi_diff, "NDWI Difference", ndwi_cmap),
                     (ndbi_diff, "NDBI Difference", ndbi_cmap),
                     (evi_diff, "EVI Difference", evi_cmap),
                     (sam_map, "SAM Map", sam_cmap),
                     (cva_map, "CVA Map", cva_cmap)]
        label_mapping = {
            "NDVI Difference": ("Vegetation\n Increase", "Vegetation\n Decrease"),
            "NDWI Difference": ("Water\n Increase", "Water\n Decrease"),
            "NDBI Difference": ("Urban\n Expansion", "Urban\n Reduction"),
            "EVI Difference": ("EVI\n Increase", "EVI\n Decrease"),
            "SAM Map": ("High\n Spectral\n Change", "Low\n Spectral\n Change"),
            "CVA Map": ("Higher\n Changes", "Lower\n Changes")
        }
        for i, (diff, title, cmap) in enumerate(diff_maps):
            row, col = divmod(i, 3)
            im = axes[row, col].imshow(diff, cmap=cmap)
            axes[row, col].contour(diff, colors='black', linewidths=0.5)
            axes[row, col].set_title(title)
            axes[row, col].axis('off')
            cbar = fig.colorbar(im, ax=axes[row, col], fraction=0.046, pad=0.04)
            top_label, bottom_label = label_mapping.get(title, ("Increase", "Decrease"))
            cbar.ax.text(0.5, 1.05, top_label, ha='center', va='bottom', fontsize=FONT_SIZE_SMALL, transform=cbar.ax.transAxes)
            cbar.ax.text(0.5, -0.1, bottom_label, ha='center', va='top', fontsize=FONT_SIZE_SMALL, transform=cbar.ax.transAxes)
        plt.tight_layout()
        plt.show()

    @staticmethod
    def display_annotated(rgb_image, legend_entries, boundaries):
        import matplotlib.pyplot as plt
        Visualizer.professional_plot_setup()
        fig, axes = plt.subplots(1, 2, figsize=FIGURE_SIZE_COMMON, gridspec_kw={'width_ratios': [1, 1]})
        ax_img = axes[0]
        ax_img.imshow(rgb_image)
        ax_img.set_title("Annotated Changes")
        ax_img.axis('off')
        for contour in boundaries:
            ax_img.plot(contour[:, 1], contour[:, 0], color='red', linewidth=0.8)
        for (num, y, x, desc) in legend_entries:
            ax_img.text(x, y, str(num), color='white', fontsize=FONT_SIZE_SMALL,
                        weight='bold', ha='center', va='center',
                        bbox=dict(facecolor='black', alpha=0.7, boxstyle='round'))
        ax_txt = axes[1]
        ax_txt.axis('off')
        detailed_legend = "\n".join([f"Region {num}: {desc}" for (num, _, _, desc) in legend_entries])
        detailed_legend += "\n\nNote: Red contours indicate detected change boundaries."
        ax_txt.text(0.5, 0.5, detailed_legend, fontsize=FONT_SIZE_SMALL, ha='center', va='center', bbox=dict(facecolor='white', alpha=0.8))
        plt.tight_layout()
        plt.show()

    @staticmethod
    def save_report_as_csv(report_rows, filename="change_detection_report.csv"):
        df = pd.DataFrame(report_rows)
        df.to_csv(filename, index=False)

    @staticmethod
    def display_report_table(report_rows):
        import matplotlib.pyplot as plt
        Visualizer.professional_plot_setup()
        df = pd.DataFrame(report_rows)
        df = df[["Region", "Centroid", "Area", "NDVI_diff", "NDWI_diff", "NDBI_diff",
                 "EVI_diff", "SAM_avg", "CVA_avg", "Veg Change", "Urban Change", "Water Change"]]
        fig, ax = plt.subplots(figsize=(18, len(df)*0.5 + 2))
        ax.axis('tight')
        ax.axis('off')
        table = ax.table(cellText=df.round(3).values,
                         colLabels=df.columns,
                         cellLoc='center',
                         loc='center')
        table.auto_set_font_size(False)
        table.set_fontsize(TABLE_FONT_SIZE)
        table.scale(1, 1.5)
        plt.title("Detailed Change Report", fontsize=TITLE_SIZE, fontweight='bold')
        plt.tight_layout()
        plt.show()

# =============================================================================
# Composite Change Index and SVM Classifier Functions
# =============================================================================
def normalize_metric(metric):
    """Normalize using min–max scaling."""
    mmin = np.min(metric)
    mmax = np.max(metric)
    if mmax - mmin == 0:
        return np.zeros_like(metric)
    return (metric - mmin) / (mmax - mmin)

def compute_composite_index(ndvi_diff, ndwi_diff, ndbi_diff, evi_diff, sam, cva,
                            weights={'ndvi':0.25, 'ndwi':0.20, 'ndbi':0.20, 'evi':0.10, 'sam':0.15, 'cva':0.10}):
    # Normalize each metric.
    ndvi_norm = normalize_metric(ndvi_diff)
    ndwi_norm = normalize_metric(ndwi_diff)
    ndbi_norm = normalize_metric(ndbi_diff)
    evi_norm  = normalize_metric(evi_diff)
    sam_norm  = normalize_metric(sam)
    cva_norm  = normalize_metric(cva)
    composite = (weights['ndvi'] * ndvi_norm +
                 weights['ndwi'] * ndwi_norm +
                 weights['ndbi'] * ndbi_norm +
                 weights['evi']  * evi_norm  +
                 weights['sam']  * sam_norm  +
                 weights['cva']  * cva_norm)
    return composite

def train_svm_classifier(features, labels):
    """
    Train an SVM classifier.
    In practice, use ground-truth data. Here, we simulate with provided features and labels.
    """
    svm = SVC(kernel='rbf', probability=True, random_state=42)
    svm.fit(features, labels)
    return svm

# =============================================================================
# Hyperparameter Tuning Function using GridSearchCV
# =============================================================================
def tune_svm_hyperparameters(features, labels):
    """
    Tune SVM hyperparameters (C and gamma for the RBF kernel) using GridSearchCV.
    Returns the best estimator and best parameters.
    """
    param_grid = {
        'C': [0.1, 1, 10, 50, 100],
        'gamma': [1e-2,1e-3, 1e-4, 1e-5, 1e-6],
        'kernel': ['rbf', 'linear', 'polynomial']
    }
    svc = SVC(probability=True, random_state=42)
    grid_search = GridSearchCV(svc, param_grid, cv=5, scoring='accuracy')
    grid_search.fit(features, labels)
    print("Best SVM Parameters:", grid_search.best_params_)
    return grid_search.best_estimator_

# =============================================================================
# ChangeDetectionPipeline Class
# =============================================================================
class ChangeDetectionPipeline:
    """
    Executes the complete change detection pipeline.
    Steps:
      1. Load and preprocess images.
      2. Compute difference maps (NDVI, NDWI, NDBI, EVI).
      3. Compute SAM and CVA.
      4. Generate global Otsu binary masks for Vegetation, Urban, and Water changes.
      5. Generate a composite change index (CCI) from all metrics.
      6. Train an SVM classifier (using synthetic training data) on the feature set.
      7. Predict change type on every pixel.
      8. Display binary masks for each change type, a combined SAM+CVA mask, and SVM classification map.
      9. Refine regions, extract boundaries, and generate a detailed report.
    """
    def __init__(self, img1_path, img2_path, threshold_scaler=THRESHOLD_SCALER,
                 min_area=MIN_REGION_AREA, apply_median=APPLY_MEDIAN, filter_radius=FILTER_RADIUS,
                 param_csv_save=SAVE_CSV, param_csv_filename=CSV_FILENAME):
        self.img1_path = img1_path
        self.img2_path = img2_path
        self.threshold_scaler = threshold_scaler
        self.min_area = min_area
        self.preproc = Preprocessor(apply_median=apply_median, filter_radius=filter_radius)
        self.detector = ChangeDetector(threshold_scaler)
        self.refiner = RegionRefiner(min_size=min_area)
        self.visualizer = Visualizer()
        self.param_csv_save = param_csv_save
        self.param_csv_filename = param_csv_filename

    def generate_report(self, refined_mask, ndvi_diff, ndwi_diff, ndbi_diff, evi_diff, sam_map, cva_map):
        report = []
        for region in regionprops(refined_mask):
            if region.area < self.min_area:
                continue
            region_mask = (refined_mask == region.label)
            centroid = region.centroid
            area = region.area
            ndvi_mean = np.mean(ndvi_diff[region_mask])
            ndwi_mean = np.mean(ndwi_diff[region_mask])
            ndbi_mean = np.mean(ndbi_diff[region_mask])
            evi_mean = np.mean(evi_diff[region_mask])
            sam_mean = np.mean(sam_map[region_mask])
            cva_mean = np.mean(cva_map[region_mask])
            desc_veg   = self.detector.generate_description('Vegetation', ndvi_mean)
            desc_urban = self.detector.generate_description('Urban', ndbi_mean)
            desc_water = self.detector.generate_description('Water', ndwi_mean)
            report.append({
                "Region": region.label,
                "Centroid": f"({centroid[0]:.1f}, {centroid[1]:.1f})",
                "Area": area,
                "NDVI_diff": ndvi_mean,
                "NDWI_diff": ndwi_mean,
                "NDBI_diff": ndbi_mean,
                "EVI_diff": evi_mean,
                "SAM_avg": sam_mean,
                "CVA_avg": cva_mean,
                "Veg Change": desc_veg,
                "Urban Change": desc_urban,
                "Water Change": desc_water
            })
        if self.param_csv_save and report:
            with open(self.param_csv_filename, mode='w', newline='') as csv_file:
                fieldnames = report[0].keys()
                writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
                writer.writeheader()
                for row in report:
                    writer.writerow(row)
        return report

    def run(self):
        # Load and preprocess images.
        image1 = self.preproc.load_image(self.img1_path)
        image2 = self.preproc.load_image(self.img2_path)
        image1 = self.preproc.preprocess(image1)
        image2 = self.preproc.preprocess(image2)
        rgb_before = self.preproc.create_rgb(image1)
        rgb_after  = self.preproc.create_rgb(image2)

        # Compute difference indices.
        ndvi1 = self.detector.compute_ndvi(image1, red_index=3, nir_index=7)
        ndvi2 = self.detector.compute_ndvi(image2, red_index=3, nir_index=7)
        ndvi_diff = ndvi2 - ndvi1

        ndwi1 = self.detector.compute_ndwi(image1, green_index=2, nir_index=7)
        ndwi2 = self.detector.compute_ndwi(image2, green_index=2, nir_index=7)
        ndwi_diff = ndwi2 - ndwi1

        ndbi1 = self.detector.compute_ndbi(image1, swir_index=11, nir_index=7)
        ndbi2 = self.detector.compute_ndbi(image2, swir_index=11, nir_index=7)
        ndbi_diff = ndbi2 - ndbi1

        evi1 = self.detector.compute_evi(image1, red_index=3, nir_index=7, blue_index=1)
        evi2 = self.detector.compute_evi(image2, red_index=3, nir_index=7, blue_index=1)
        evi_diff = evi2 - evi1

        # Compute SAM and CVA maps (using Vegetation bands).
        bands = ChangeDetector.selected_bands['Vegetation']
        sam_map = self.detector.compute_sam(image1, image2, bands)
        cva_map = self.detector.compute_cva(image1, image2, bands)

        # Generate binary masks via global Otsu thresholding using the same method (no differentiation).
        binary_veg = self.detector.apply_otsu(ndvi_diff)
        binary_urban = self.detector.apply_otsu(ndbi_diff)
        binary_water = self.detector.apply_otsu(ndwi_diff)

        # Display the three binary masks side by side.
        fig, axes = plt.subplots(1, 3, figsize=(15, 5))
        from matplotlib.colors import ListedColormap
        import matplotlib.patches as mpatches

        veg_cmap = ListedColormap(["black", "green"])
        axes[0].imshow(binary_veg, cmap=veg_cmap)
        axes[0].set_title("Vegetation Change Mask")
        axes[0].axis('off')
        patch_none = mpatches.Patch(color="black", label="No Change")
        patch_veg = mpatches.Patch(color="green", label="Vegetation Change")
        axes[0].legend(handles=[patch_none, patch_veg], loc='lower right')

        urban_cmap = ListedColormap(["black", "red"])
        axes[1].imshow(binary_urban, cmap=urban_cmap)
        axes[1].set_title("Urban Change Mask")
        axes[1].axis('off')
        patch_urban = mpatches.Patch(color="red", label="Urban Change")
        axes[1].legend(handles=[patch_none, patch_urban], loc='lower right')

        water_cmap = ListedColormap(["black", "blue"])
        axes[2].imshow(binary_water, cmap=water_cmap)
        axes[2].set_title("Water Change Mask")
        axes[2].axis('off')
        patch_water = mpatches.Patch(color="blue", label="Water Change")
        axes[2].legend(handles=[patch_none, patch_water], loc='lower right')
        plt.tight_layout()
        plt.show()

        # Generate a binary mask from SAM and CVA using global Otsu (using same calculation for both)
        binary_sam = self.detector.apply_otsu(sam_map)
        binary_cva = self.detector.apply_otsu(cva_map)
        binary_sam_cva = np.logical_or(binary_sam, binary_cva)
        fig, ax = plt.subplots(figsize=FIGURE_SIZE_COMMON)
        mask_cmap = ListedColormap(["black", "yellow"])
        ax.imshow(binary_sam_cva, cmap=mask_cmap)
        ax.set_title("SAM & CVA Combined Change Mask")
        ax.axis('off')
        patch_samcva_none = mpatches.Patch(color="black", label="No Change")
        patch_samcva_yes = mpatches.Patch(color="yellow", label="Change")
        ax.legend(handles=[patch_samcva_none, patch_samcva_yes], loc='lower right')
        plt.tight_layout()
        plt.show()

        # Compute Composite Change Index (CCI)
        composite_index = compute_composite_index(ndvi_diff, ndwi_diff, ndbi_diff, evi_diff, sam_map, cva_map)
        # For demonstration, threshold CCI at the 75th percentile.
        threshold_cci = np.percentile(composite_index, 75)
        binary_cci = composite_index > threshold_cci
        fig, ax = plt.subplots(figsize=FIGURE_SIZE_COMMON)
        ax.imshow(binary_cci, cmap=ListedColormap(["black", "magenta"]))
        ax.set_title("Composite Change Index (CCI) Binary Map")
        ax.axis('off')
        patch_cci_none = mpatches.Patch(color="black", label="No Change")
        patch_cci_yes = mpatches.Patch(color="magenta", label="Change")
        ax.legend(handles=[patch_cci_none, patch_cci_yes], loc='lower right')
        plt.tight_layout()
        plt.show()

        # Create feature set for SVM (flatten all feature images)
        # Features: NDVI_diff, NDWI_diff, NDBI_diff, EVI_diff, SAM, CVA
        feat_ndvi = ndvi_diff.flatten()
        feat_ndwi = ndwi_diff.flatten()
        feat_ndbi = ndbi_diff.flatten()
        feat_evi  = evi_diff.flatten()
        feat_sam  = sam_map.flatten()
        feat_cva  = cva_map.flatten()
        features = np.column_stack((feat_ndvi, feat_ndwi, feat_ndbi, feat_evi, feat_sam, feat_cva))

        # For demonstration, create synthetic training labels using simple rules.
        # (In practice, use ground-truth training data.)
        # Rules: If NDVI_diff < -0.1 then Vegetation loss (label 1), if NDBI_diff > 0.1 then Urban expansion (label 2),
        # if NDWI_diff > 0.1 then Water increase (label 3), else 0 (no change).

        #---------------------------
        # labels = np.zeros(features.shape[0], dtype=int)
        # labels[feat_ndvi < -0.1] = 1
        # labels[feat_ndbi > 0.1] = 2
        # labels[feat_ndwi > 0.1] = 3
        #------------
                # Create synthetic training labels using adjusted thresholds based on literature guidelines.
        labels = np.zeros(features.shape[0], dtype=int)
        # NDVI: If the difference is less than -0.15, consider it a vegetation loss event.
        labels[feat_ndvi < -0.05] = 1
        # NDBI: If the difference is greater than 0.15, label it as urban expansion.
        labels[feat_ndbi > 0.30] = 2
        # NDWI: If the difference is greater than 0.12, label it as water increase.
        labels[feat_ndwi > 0.12] = 3


        # Normalize features.
        scaler = MinMaxScaler()
        features_norm = scaler.fit_transform(features)

        # Optionally, you can call the hyperparameter tuning function here.
        # Uncomment the next two lines to perform GridSearchCV and use the best estimator.
        # best_svm = tune_svm_hyperparameters(features_norm, labels)
        # svm_classifier = best_svm

        # Otherwise, train the SVM with default hyperparameters.
        svm_classifier = SVC(C=100, gamma=0.01, kernel='linear', probability=True, random_state=42)
        svm_classifier.fit(features_norm, labels)
        print("SVM classifier trained on synthetic data.")

        # Predict on entire image.
        svm_pred = svm_classifier.predict(features_norm)
        svm_map = svm_pred.reshape(ndvi_diff.shape)

        # Display SVM classification map.
        cmap_svm = plt.cm.get_cmap('viridis', 4)  # 4 classes: 0,1,2,3
        plt.figure(figsize=FIGURE_SIZE_COMMON)
        plt.imshow(svm_map, cmap=cmap_svm)
        plt.title("SVM Classification Map (0: No Change, 1: Vegetation, 2: Urban, 3: Water)")
        plt.axis('off')
        plt.colorbar(ticks=range(4))
        plt.tight_layout()
        plt.show()

        # ---------------------------------------------------------------------
        # Additional Difference Plots with Masked Contours
        # ---------------------------------------------------------------------
        # Plot 1: NDVI, NDWI, NDBI difference maps with contours only where SAM+CVA binary mask is true.
        fig, axes = plt.subplots(1, 3, figsize=FIGURE_SIZE_COMMON)
        for idx, (diff, title, cmap) in enumerate([(ndvi_diff, "NDVI Diff masked by SAM+CVA", ndvi_cmap),
                                                   (ndwi_diff, "NDWI Diff masked by SAM+CVA", ndwi_cmap),
                                                   (ndbi_diff, "NDBI Diff masked by SAM+CVA", ndbi_cmap)]):
            ax = axes[idx]
            im = ax.imshow(diff, cmap=cmap)
            # Only plot contours where binary_sam_cva is True.
            masked_diff = np.where(binary_sam_cva, diff, np.nan)
            ax.contour(masked_diff, colors='black', linewidths=0.5)
            ax.set_title(title)
            ax.axis('off')
            fig.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
        plt.tight_layout()
        plt.show()

        # Plot 2: NDVI, NDWI, NDBI difference maps with contours only where CCI binary mask is true.
        fig, axes = plt.subplots(1, 3, figsize=FIGURE_SIZE_COMMON)
        for idx, (diff, title, cmap) in enumerate([(ndvi_diff, "NDVI Diff masked by CCI", ndvi_cmap),
                                                   (ndwi_diff, "NDWI Diff masked by CCI", ndwi_cmap),
                                                   (ndbi_diff, "NDBI Diff masked by CCI", ndbi_cmap)]):
            ax = axes[idx]
            im = ax.imshow(diff, cmap=cmap)
            # Only plot contours where binary_cci is True.
            masked_diff = np.where(binary_cci, diff, np.nan)
            ax.contour(masked_diff, colors='black', linewidths=0.5)
            ax.set_title(title)
            ax.axis('off')
            fig.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
        plt.tight_layout()
        plt.show()
        # ---------------------------------------------------------------------

        # Refine regions and extract boundaries for one example (vegetation).
        # Process vegetation
        refine_yo = True
        if refine_yo:
            refined_veg = self.refiner.refine(binary_veg)
            boundaries_veg = self.refiner.extract_boundaries(refined_veg)
        else:
            boundaries_veg = self.refiner.extract_boundaries(binary_veg)

        # Process urban
        if refine_yo:
            refined_urban = self.refiner.refine(binary_urban)
            boundaries_urban = self.refiner.extract_boundaries(refined_urban)
        else:
            boundaries_urban = self.refiner.extract_boundaries(binary_urban)

        # Process water
        if refine_yo:
            refined_water = self.refiner.refine(binary_water)
            boundaries_water = self.refiner.extract_boundaries(refined_water)
        else:
            boundaries_water = self.refiner.extract_boundaries(binary_water)

        # Combine boundaries from all masks
        combined_boundaries = boundaries_veg + boundaries_urban + boundaries_water

        # Display before/after composites.
        self.visualizer.display_before_after(rgb_before, rgb_after)
        # Display difference maps.
        self.visualizer.display_difference_maps(ndvi_diff, ndwi_diff, ndbi_diff, evi_diff, sam_map, cva_map)
        # Annotate changes (using refined vegetation regions as example).
        legend_entries = []
        region_number = 1
        for region in regionprops(refined_veg):
            if region.area < self.min_area:
                continue
            cy, cx = region.centroid
            avg_sam = np.mean(sam_map[refined_veg == region.label])
            desc = f"SAM avg: {avg_sam:.2f} rad, Centroid: ({cy:.1f},{cx:.1f}), Area: {region.area}"
            legend_entries.append((region_number, cy, cx, desc))
            region_number += 1
        self.visualizer.display_annotated(rgb_after, legend_entries, combined_boundaries)

        # Generate and display detailed change report.
        report = self.generate_report(refined_veg, ndvi_diff, ndwi_diff, ndbi_diff, evi_diff, sam_map, cva_map)
        print("\nDetailed Change Report:")
        self.visualizer.display_report_table(report)
        print("Pipeline complete.")

  # =============================================================================
# Main execution block
# =============================================================================
if __name__ == "__main__":

    pipeline = ChangeDetectionPipeline(
        FILE_PATH_BEFORE,
        FILE_PATH_AFTER,
        threshold_scaler=THRESHOLD_SCALER,
        min_area=MIN_REGION_AREA,
        apply_median=APPLY_MEDIAN,
        filter_radius=FILTER_RADIUS,
        param_csv_save=SAVE_CSV,
        param_csv_filename=CSV_FILENAME
    )
    # Normalize features (features_norm) and then:

    pipeline.run()
